## Extract all fields and check which aren't mapped

### Usage

Run All

Output will give two lists for each form:

- missingV1Mappings
  - Fields in the v1 form which do not have a mapping in v2
- extraV2Mappings
  - Fields in the v2 form which do not exist in v1

Any country specific mappings can be added to `helpers/countryMappings.ts`


In [1]:
import { authenticate } from './helpers/authentication.ts'

const token = await authenticate(GATEWAY, 's.faisal', 'test')
token


"eyJhbGciOiJSUzI1NiIsInR5cCI6IkpXVCJ9.eyJzY29wZSI6WyJyZWNvcmQuZGVjbGFyZS1iaXJ0aCIsInJlY29yZC5kZWNsYXJlLWRlYXRoIiwicmVjb3JkLmRlY2xhcmUtbWFycmlhZ2UiLCJyZWNvcmQuZGVjbGFyYXRpb24tZWRpdCIsInJlY29yZC5kZWNsYXJhdGlvbi1zdWJtaXQtZm9yLXVwZGF0ZXMiLCJyZWNvcmQucmV2aWV3LWR1cGxpY2F0ZXMiLCJyZWNvcmQuZGVjbGFyYXRpb24tYXJjaGl2ZSIsInJlY29yZC5kZWNsYXJhdGlvbi1yZWluc3RhdGUiLCJyZWNvcmQucmVnaXN0ZXIiLCJyZWNvcmQucmVnaXN0cmF0aW9uLWNvcnJlY3QiLCJyZWNvcmQuZGVjbGFyYXRpb24tcHJpbnQtc3VwcG9ydGluZy1kb2N1bWVudHMiLCJyZWNvcmQuZXhwb3J0LXJlY29yZHMiLCJyZWNvcmQudW5hc3NpZ24tb3RoZXJzIiwicmVjb3JkLnJlZ2lzdHJhdGlvbi1wcmludCZpc3N1ZS1jZXJ0aWZpZWQtY29waWVzIiwicmVjb3JkLmNvbmZpcm0tcmVnaXN0cmF0aW9uIiwicmVjb3JkLnJlamVjdC1yZWdpc3RyYXRpb24iLCJwZXJmb3JtYW5jZS5yZWFkIiwicGVyZm9ybWFuY2UucmVhZC1kYXNoYm9hcmRzIiwicGVyZm9ybWFuY2Uudml0YWwtc3RhdGlzdGljcy1leHBvcnQiLCJwcm9maWxlLmVsZWN0cm9uaWMtc2lnbmF0dXJlIiwib3JnYW5pc2F0aW9uLnJlYWQtbG9jYXRpb25zOm15LW9mZmljZSIsInNlYXJjaC5iaXJ0aCIsInNlYXJjaC5kZWF0aCIsInNlYXJjaC5tYXJyaWFnZSIsInVzZXIucmVhZDpteS1vZmZpY2UiLCJzZW

In [2]:
import { fetchEvents, fetchForms } from './helpers/formsHandlers.ts'

const forms = await fetchForms(token)
const events = await fetchEvents(token)

Deno.writeFileSync(
  './countryData/v1Forms.json',
  new TextEncoder().encode(JSON.stringify(forms, null, 2)))

In [3]:
import { extractFieldType, extractFormFields } from './helpers/utils.ts'

const ignoredFields = [
  'DIVIDER',
  'PARAGRAPH',
  'BULLET_LIST',
  'HEADING3',
  'SIGNATURE',
]

const birthFormFields = extractFormFields(forms, 'birth')
  .filter((x) => !ignoredFields.includes(x.type))
  .map((f) => f.id)

const deathFormFields = extractFormFields(forms, 'death')
  .filter((x) => !ignoredFields.includes(x.type))
  .map((f) => f.id)

const birthEvent = events.find((x) => x.id === 'birth')
const deathEvent = events.find((x) => x.id === 'death')

const birthEventFields = [
  ...new Set(
    extractFieldType(birthEvent, 'fields')
      .filter((x) => !ignoredFields.includes(x.type))
      .map((f) => f.id)
      .filter((x) => x)
  ),
]

const deathEventFields = [
  ...new Set(
    extractFieldType(deathEvent, 'fields')
      .filter((x) => !ignoredFields.includes(x.type))
      .map((f) => f.id)
      .filter((x) => x)
  ),
]


[
  "birth.child-view-group.firstNamesEng",
  "birth.child-view-group.familyNameEng",
  "birth.child-view-group.childBirthDate",
  "birth.child-view-group.gender",
  "birth.child-view-group.iD",
  "birth.child-view-group.placeOfBirth",
  "birth.child-view-group.birthLocation",
  "birth.child-view-group.countryPlaceofbirth",
  "birth.child-view-group.statePlaceofbirth",
  "birth.child-view-group.districtPlaceofbirth",
  "birth.child-view-group.cityPlaceofbirth",
  "birth.child-view-group.addressLine1Placeofbirth",
  "birth.child-view-group.addressLine2Placeofbirth",
  "birth.child-view-group.addressLine3Placeofbirth",
  "birth.child-view-group.postalCodePlaceofbirth",
  "birth.child-view-group.internationalCityPlaceofbirth",
  "birth.child-view-group.countryPrimaryChild",
  "birth.child-view-group.statePrimaryChild",
  "birth.child-view-group.districtPrimaryChild",
  "birth.child-view-group.cityPrimaryChild",
  "birth.child-view-group.addressLine1PrimaryChild",
  "birth.child-view-group

In [4]:
import {
  DEFAULT_FIELD_MAPPINGS,
  CUSTOM_FIELD_MAPPINGS,
} from './helpers/defaultMappings.ts'
import { ADDRESS_MAPPINGS } from './countryData/addressMappings.ts'
import { NAME_MAPPINGS } from './countryData/nameMappings.ts'
import { COUNTRY_FIELD_MAPPINGS } from './countryData/countryMappings.ts'
import defaultResolver from './helpers/defaultResolvers.ts'
import { countryResolver } from './countryData/countryResolvers.ts'
import historyResolver from './helpers/historyResolver.ts'
import { v1IgnoreList } from './countryData/ignoreList.ts'

const resolver = { ...defaultResolver, ...countryResolver }

const addressMap = Object.fromEntries(
  Object.entries(ADDRESS_MAPPINGS).map(
    ([f, value]) => [f, Object.keys(value(null))[0]]
  )
)

const nameMap = Object.fromEntries(
  Object.entries(NAME_MAPPINGS).map(
    ([f, value]) => [f, Object.keys(value(null))[0]]
  )
)

const allMappings = {
  ...DEFAULT_FIELD_MAPPINGS,
  ...CUSTOM_FIELD_MAPPINGS,
  ...addressMap,
  ...nameMap,
  ...COUNTRY_FIELD_MAPPINGS,
}

function partition<T>(list: T[], predicate: (item: T) => boolean) {
  const map: T = {}
  const unmapped: T[] = []

  for (const i of list) {
    if (predicate(i)) {
      map[i] = predicate(i)
    } else {
      unmapped.push(i)
    }
  }

  return { map, unmapped }
}

const birthFormMappings = partition(birthFormFields.filter(f => !v1IgnoreList.includes(f)), (f) => allMappings[f])
const deathFormMappings = partition(deathFormFields.filter(f => !v1IgnoreList.includes(f)), (f) => allMappings[f])

const unMappedBirthEventFields = birthEventFields.filter(
  (f) => !Object.values(birthFormMappings.map).includes(f) && !Object.keys(historyResolver).includes(f)
)
const unMappedDeathEventFields = deathEventFields.filter(
  (f) => !Object.values(deathFormMappings.map).includes(f) && !Object.keys(historyResolver).includes(f)
)

const badBirthMappings = Object.entries(birthFormMappings.map).filter(
  ([_k, v]) => !birthEventFields.includes(v)
)

const badDeathMappings = Object.entries(deathFormMappings.map).filter(
  ([_k, v]) => !deathEventFields.includes(v)
)

const unmappedFields = {
  birth: {
    missingV1Mappings: birthFormMappings.unmapped,
    extraV2Mappings: unMappedBirthEventFields,
  },
  death: {
    missingV1Mappings: deathFormMappings.unmapped,
    extraV2Mappings: unMappedDeathEventFields,
  },
}

const unResolvedBirthFields = Object.entries(birthFormMappings.map)
  .filter(([_k, v]) => !badBirthMappings.some(([_k1, v1]) => v1 === v))
  .filter(([_k, v]) => !resolver[v])

const unResolvedDeathFields = Object.entries(deathFormMappings.map)
  .filter(([_k, v]) => !badDeathMappings.some(([_k1, v1]) => v1 === v))
  .filter(([_k, v]) => !resolver[v])


In [5]:
const render = (list) => {
  if (list.length) {
    console.table(list)
  } else {
    console.log('None \x1b[32m✔\x1b[0m')
  }
}

console.log('Unmapped V1 form fields:')
console.log()

console.log('Birth:')
render(unmappedFields.birth.missingV1Mappings)

console.log('Death:')
render(unmappedFields.death.missingV1Mappings)
console.log()
console.log('V1 form fields mapped to a V2 field that does not exist:')
console.log('Birth:')
render(badBirthMappings.map(([k, v]) => ({ 'V1 Field': k, 'V2 Field': v })))

console.log('Death:')
render(badDeathMappings.map(([k, v]) => ({ 'V1 Field': k, 'V2 Field': v })))

console.log()
console.log('V1 form fields mapped but V2 field does not have a resolver:')
console.log('Birth:')
render(
  unResolvedBirthFields.map(([k, v]) => ({ 'V1 Field': k, 'V2 Field': v }))
)
console.log('Death:')
render(
  unResolvedDeathFields.map(([k, v]) => ({ 'V1 Field': k, 'V2 Field': v }))
)

console.log()
console.log('Additional V2 form fields not mapped:')
console.log('Birth:')
render(unmappedFields.birth.extraV2Mappings)

console.log('Death:')
render(unmappedFields.death.extraV2Mappings)

if (
  unmappedFields.birth.missingV1Mappings.length ||
  unmappedFields.death.missingV1Mappings.length ||
  badBirthMappings.length ||
  badDeathMappings.length ||
  unResolvedBirthFields.length ||
  unResolvedDeathFields.length
) {
  throw new Error('Migration may not proceed. Please fix errors')
} else {
  console.log()
  console.log('Ok to proceed with migration')
}


Unmapped V1 form fields:

Birth:
None ✔
Death:
None ✔

V1 form fields mapped to a V2 field that does not exist:
Birth:
None ✔
Death:
None ✔

V1 form fields mapped but V2 field does not have a resolver:
Birth:
┌───────┬────────────────────────────────────────────────────────┬───────────────────────────┐
│ (idx) │ V1 Field                                               │ V2 Field                  │
├───────┼────────────────────────────────────────────────────────┼───────────────────────────┤
│     0 │ "birth.child-view-group.iD"                            │ "child.nid"               │
│     1 │ "birth.child-view-group.countryPrimaryChild"           │ "child.address"           │
│     2 │ "birth.child-view-group.statePrimaryChild"             │ "child.address"           │
│     3 │ "birth.child-view-group.districtPrimaryChild"          │ "child.address"           │
│     4 │ "birth.child-view-group.cityPrimaryChild"              │ "child.address"           │
│     5 │ "birth.child-view-gro

Error: Migration may not proceed. Please fix errors